In [1]:
# importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

#prevent oom
x_train = x_train[:10000]
y_train = y_train[:10000]

# Normalize images to the range [0, 1]
x_train = (x_train / 255.0).astype(np.float32)
x_test = (x_test / 255.0).astype(np.float32)

# Flatten the images to be vectors
x_train = x_train.reshape(-1, 28 * 28).astype(np.float32)
x_test = x_test.reshape(-1, 28 * 28).astype(np.float32)

# Convert the digit labels to even/odd labels:
# Even -> 0, Odd -> 1
y_train_even_odd = np.array([label % 2 for label in y_train], dtype=np.int32)
y_test_even_odd = np.array([label % 2 for label in y_test], dtype=np.int32)


In [2]:
import tensorflow as tf
device = "/GPU:0" if tf.config.list_physical_devices('GPU') else "/CPU:0"



In [3]:
class CustomDenseLayerTF:
    def __init__(self, num_inputs, num_neurons):
        with tf.device(device):  # to run the operation on gpu
            #defining the custom wegitshs and Gweights along with bias
            self.weights = tf.Variable(tf.random.normal([num_inputs, num_neurons], stddev=0.01, dtype=tf.float32))
            self.Gweights = tf.Variable(tf.random.normal([num_inputs, num_neurons], stddev=0.01, dtype=tf.float32))
            self.bias = tf.Variable(tf.zeros([1, num_neurons], dtype=tf.float32))

    def forward(self, inputs):
        """
        Performs forward propgation of the dense neuraon
        Params: 
        inputs : input from the previous layer or the input layer
        Returns:
        output : output of the dense
        """
        self.inputs = inputs
        self.output = tf.matmul(inputs, self.weights) + tf.matmul(inputs, self.Gweights) + self.bias

    def update_weights(self, gradients, learning_rate=0.01):
        """
        Updates the weights of the dense layer
        Params: 
        gradients : gradients of the weights and bias
        learning_rate : learning rate for the optimizer
        """
        self.weights.assign_sub(learning_rate * gradients[0])  # Update weights
        self.Gweights.assign_sub(learning_rate * gradients[1])  # Update weights
        self.bias.assign_sub(learning_rate * gradients[2])      # Update bias
        

In [4]:
#defining the activation functions
class ActivationSigmoidTF:
    def forward(self, inputs):
        """
        perform forwrad pass for the sigmoid activation function
        Params:
        inputs : input from the previous layer
        Returns:    
        output : output of the sigmoid activation function
        """
        self.output = tf.nn.sigmoid(inputs)

class LossBinaryCrossentropyTF:
    def calculate(self, output, y_true):
        """
        Caclulates the binary crossentropy loss
        Params:
        output : output of the model    
        y_true : true labels
        Returns:
        loss : binary crossentropy loss
        """
        output = tf.clip_by_value(output, 1e-7, 1 - 1e-7)  # Avoid log(0)
        return tf.reduce_mean(- (y_true * tf.math.log(output) + (1 - y_true) * tf.math.log(1 - output)))


In [5]:
# training function 
@tf.function  # Compiles function for efficiency (Graph Mode)
def train_step(dense_layer, activation, loss_function, X_batch, y_batch, optimizer):
    with tf.GradientTape() as tape:
        # Forward pass of the custom dense network
        dense_layer.forward(X_batch)
        # incorporating the activation function 
        activation.forward(dense_layer.output)
        #calculating the loss values
        loss = loss_function.calculate(activation.output, y_batch)

    # Compute gradients
    gradients = tape.gradient(loss, [dense_layer.weights, dense_layer.Gweights,  dense_layer.bias])

    # Ensure valid gradients
    if gradients is None or any(g is None for g in gradients):
        return None  # If gradient calculation fails, return None

    # Apply gradients
    optimizer.apply_gradients(zip(gradients, [dense_layer.weights, dense_layer.Gweights, dense_layer.bias]))

    # Compute accuracy
    predictions = tf.cast(activation.output > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, y_batch), dtype=tf.float32))

    return loss, accuracy

In [6]:
#defining the main training loop:
def train_custom_nn(X_train, y_train, epochs=5, batch_size=32, learning_rate=0.01):
    num_samples = X_train.shape[0]

    # Convert input data & labels to TensorFlow tensors
    X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_train_tf = tf.convert_to_tensor(y_train.reshape(-1, 1), dtype=tf.float32)

    # Create TensorFlow dataset for efficient training
    dataset = tf.data.Dataset.from_tensor_slices((X_train_tf, y_train_tf))
    dataset = dataset.shuffle(num_samples).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    # Initialize custom model
    with tf.device(device):
        dense_layer = CustomDenseLayerTF(X_train.shape[1], 1)
        activation = ActivationSigmoidTF()
        loss_function = LossBinaryCrossentropyTF()
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Adam optimizer

    # Training loop
    for epoch in range(epochs):
        total_loss = tf.Variable(0.0, dtype=tf.float32)
        total_accuracy = tf.Variable(0.0, dtype=tf.float32)
        num_batches = tf.Variable(0, dtype=tf.int32)

        # Process data in batches using TensorFlow dataset
        for X_batch, y_batch in dataset:
            result = train_step(dense_layer, activation, loss_function, X_batch, y_batch, optimizer)
            
            if result is None:
                continue  # Skip this batch if train_step() returned None
            
            loss, accuracy = result
            total_loss.assign_add(loss)
            total_accuracy.assign_add(accuracy)
            num_batches.assign_add(1)

        # Compute average loss and accuracy per epoch
        avg_loss = total_loss / tf.cast(num_batches, tf.float32)
        avg_accuracy = total_accuracy / tf.cast(num_batches, tf.float32)

        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss.numpy():.4f}, Accuracy: {avg_accuracy.numpy():.4f}")
    
    return dense_layer, activation

In [7]:
# import tensorflow as tf
# import numpy as np

# # ================== CUSTOM DENSE LAYER WITH DELTA CONDITION ==================
# class CustomDenseLayerTF:
#     def __init__(self, num_inputs, num_neurons):
#         with tf.device(device):  # Ensure operations are performed on GPU if available
#             self.weights = tf.Variable(tf.random.normal([num_inputs, num_neurons], stddev=0.01, dtype=tf.float32))
#             self.Gweights = tf.Variable(tf.random.normal([num_inputs, num_neurons], stddev=0.01, dtype=tf.float32))
#             self.bias = tf.Variable(tf.zeros([1, num_neurons], dtype=tf.float32))
#             self.prev_output = None  # Store previous output for delta calculation

#     def forward(self, inputs):
#         self.inputs = inputs
#         self.output = tf.matmul(inputs, self.weights) + tf.matmul(inputs, self.Gweights) + self.bias

#     def update_weights(self, gradients, learning_rate=0.01, delta=0.5):
#         """
#         Updates weights only if |zi+1 - zi| < δ (delta).
#         """
#         if self.prev_output is not None:
#             # Ensure prev_output matches batch size
#             if self.prev_output.shape != self.output.shape:
#                 self.prev_output = tf.zeros_like(self.output)  # Reset to correct shape

#             output_diff = tf.abs(self.output - self.prev_output)  # Compute |zi+1 - zi|

#             # Mask for elements where |zi+1 - zi| < delta
#             update_mask = tf.cast(output_diff < delta, dtype=tf.float32)

#             # Reshape `update_mask` to match weights' shape
#             update_mask = tf.reshape(update_mask, [1, -1])  # [batch_size, num_neurons] -> [1, num_neurons]
#             update_mask = tf.broadcast_to(update_mask, self.weights.shape)  # Expand to match weights

#             # Apply updates only to neurons where the condition holds
#             self.weights.assign_sub(learning_rate * gradients[0] * update_mask)
#             self.Gweights.assign_sub(learning_rate * gradients[1] * update_mask)
#             self.bias.assign_sub(learning_rate * gradients[2] * update_mask)

#         # Update the previous output for the next iteration
#         self.prev_output = tf.identity(self.output)  # Ensure shape consistency 

# # ================== SIGMOID ACTIVATION FUNCTION ==================
# class ActivationSigmoidTF:
#     def forward(self, inputs):
#         self.output = tf.nn.sigmoid(inputs)


# # ================== BINARY CROSS-ENTROPY LOSS FUNCTION ==================
# class LossBinaryCrossentropyTF:
#     def calculate(self, output, y_true):
#         output = tf.clip_by_value(output, 1e-7, 1 - 1e-7)  # Avoid log(0)
#         return tf.reduce_mean(- (y_true * tf.math.log(output) + (1 - y_true) * tf.math.log(1 - output)))


# # ================== TRAINING FUNCTION WITH DELTA CONDITION ==================
# @tf.function  # Compiles function for efficiency (Graph Mode)
# def train_step(dense_layer, activation, loss_function, X_batch, y_batch, optimizer, delta):
#     with tf.GradientTape() as tape:
#         # Forward pass
#         dense_layer.forward(X_batch)
#         activation.forward(dense_layer.output)
#         loss = loss_function.calculate(activation.output, y_batch)

#     # Compute gradients
#     gradients = tape.gradient(loss, [dense_layer.weights, dense_layer.Gweights, dense_layer.bias])

#     # Ensure valid gradients
#     if gradients is None or any(g is None for g in gradients):
#         return None  # If gradient calculation fails, return None

#     # Update weights based on delta condition
#     dense_layer.update_weights(gradients, learning_rate=optimizer.learning_rate, delta=delta)

#     # Compute accuracy
#     predictions = tf.cast(activation.output > 0.5, dtype=tf.float32)
#     accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, y_batch), dtype=tf.float32))

#     return loss, accuracy


# # ================== MAIN TRAINING FUNCTION ==================
# def train_custom_nn(X_train, y_train, epochs=5, batch_size=32, learning_rate=0.01, delta=0.5):
#     num_samples = X_train.shape[0]

#     # Convert input data & labels to TensorFlow tensors
#     X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
#     y_train_tf = tf.convert_to_tensor(y_train.reshape(-1, 1), dtype=tf.float32)

#     # Create TensorFlow dataset for efficient training
#     dataset = tf.data.Dataset.from_tensor_slices((X_train_tf, y_train_tf))
#     dataset = dataset.shuffle(num_samples).batch(batch_size).prefetch(tf.data.AUTOTUNE)

#     # Initialize custom model
#     with tf.device(device):
#         dense_layer = CustomDenseLayerTF(X_train.shape[1], 1)
#         activation = ActivationSigmoidTF()
#         loss_function = LossBinaryCrossentropyTF()
#         optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Adam optimizer

#     # Training loop
#     for epoch in range(epochs):
#         total_loss = tf.Variable(0.0, dtype=tf.float32)
#         total_accuracy = tf.Variable(0.0, dtype=tf.float32)
#         num_batches = tf.Variable(0, dtype=tf.int32)

#         # Process data in batches using TensorFlow dataset
#         for X_batch, y_batch in dataset:
#             result = train_step(dense_layer, activation, loss_function, X_batch, y_batch, optimizer, delta)
            
#             if result is None:
#                 continue  # Skip this batch if train_step() returned None
            
#             loss, accuracy = result
#             total_loss.assign_add(loss)
#             total_accuracy.assign_add(accuracy)
#             num_batches.assign_add(1)

#         # Compute average loss and accuracy per epoch
#         avg_loss = total_loss / tf.cast(num_batches, tf.float32)
#         avg_accuracy = total_accuracy / tf.cast(num_batches, tf.float32)

#         print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss.numpy():.4f}, Accuracy: {avg_accuracy.numpy():.4f}")
    
#     return dense_layer, activation

In [8]:
# Train the custom neural network with a batch size of 64
dense_layer, activation = train_custom_nn(x_train, y_train_even_odd, epochs=10, batch_size=32)


2025-02-28 11:36:02.387449: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-02-28 11:36:02.387495: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-28 11:36:02.387506: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-28 11:36:02.387532: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-28 11:36:02.387553: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-02-28 11:36:02.679688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-02-28 11:36:04.269472: I t

Epoch 1/10 - Loss: 0.3047, Accuracy: 0.8745


2025-02-28 11:36:05.326053: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2/10 - Loss: 0.2753, Accuracy: 0.8907
Epoch 3/10 - Loss: 0.2672, Accuracy: 0.8975


2025-02-28 11:36:07.338235: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 4/10 - Loss: 0.2701, Accuracy: 0.8968
Epoch 5/10 - Loss: 0.2529, Accuracy: 0.9001
Epoch 6/10 - Loss: 0.2673, Accuracy: 0.8987
Epoch 7/10 - Loss: 0.2596, Accuracy: 0.8985


2025-02-28 11:36:11.397350: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 8/10 - Loss: 0.2666, Accuracy: 0.8961
Epoch 9/10 - Loss: 0.2613, Accuracy: 0.8971
Epoch 10/10 - Loss: 0.2600, Accuracy: 0.8964


### Comparing with prebuild keras model


In [23]:
from tensorflow import keras
# ================== TENSORFLOW MODEL (Using GPU) ==================

# Define the equivalent TensorFlow model
with tf.device(device):  # Run on GPU
    model = keras.Sequential([
        keras.layers.Dense(1, activation='sigmoid', input_shape=(28 * 28,))
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    # Train for 5 epochs
    model.fit(x_train, y_train_even_odd, epochs=5, verbose=1, batch_size=32)

    # Evaluate on test data
    loss_tf, accuracy_tf = model.evaluate(x_train, y_train_even_odd, verbose=0)

print("\n===== Comparison =====")
# print(f"Custom Neural Network Accuracy: {acc:.4f}")
print(f"TensorFlow Neural Network Accuracy: {accuracy_tf:.4f}")

/opt/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                 │ (None, 1)              │           785 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 785 (3.07 KB)

 Trainable params: 785 (3.07 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7165 - loss: 0.5611
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8619 - loss: 0.3317
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8781 - loss: 0.2954
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8916 - loss: 0.2757
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8930 - loss: 0.2671

===== Comparison =====
TensorFlow Neural Network Accuracy: 0.8914


In [3]:
class CustomDenseLayerTF:
    def __init__(self, num_inputs, num_neurons):
        with tf.device(device):
            self.weights = tf.Variable(tf.random.normal([num_inputs, num_neurons], stddev=0.01, dtype=tf.float32))
            self.Gweights = tf.Variable(tf.random.normal([num_inputs, num_neurons], stddev=0.01, dtype=tf.float32))
            self.bias = tf.Variable(tf.zeros([1, num_neurons], dtype=tf.float32))
            self.prev_output = None  # Store previous output (Zi)

    def forward(self, inputs, delta=0.5):
        self.inputs = inputs
        new_output = tf.matmul(inputs, self.weights) + tf.matmul(inputs, self.Gweights) + self.bias

        if self.prev_output is not None:
            output_diff = tf.abs(new_output - self.prev_output)
            mask = tf.cast(output_diff < delta, dtype=tf.float32)
            self.output = mask * new_output + (1 - mask) * self.prev_output
        else:
            self.output = new_output  # First iteration, no previous output

    def update_weights(self, gradients, learning_rate=0.01):
        """
        Updates the weights of the dense layer
        Params: 
        gradients : gradients of the weights and bias
        learning_rate : learning rate for the optimizer
        """
        self.weights.assign_sub(learning_rate * gradients[0])  # Update weights
        self.Gweights.assign_sub(learning_rate * gradients[1])  # Update weights
        self.bias.assign_sub(learning_rate * gradients[2])      # Update bias

    

In [4]:
#defining the activation functions
class ActivationSigmoidTF:
    def forward(self, inputs):
        """
        perform forwrad pass for the sigmoid activation function
        Params:
        inputs : input from the previous layer
        Returns:    
        output : output of the sigmoid activation function
        """
        self.output = tf.nn.sigmoid(inputs)

class LossBinaryCrossentropyTF:
    def calculate(self, output, y_true):
        """
        Caclulates the binary crossentropy loss
        Params:
        output : output of the model    
        y_true : true labels
        Returns:
        loss : binary crossentropy loss
        """
        output = tf.clip_by_value(output, 1e-7, 1 - 1e-7)  # Avoid log(0)
        return tf.reduce_mean(- (y_true * tf.math.log(output) + (1 - y_true) * tf.math.log(1 - output)))


In [5]:
@tf.function
def train_step(dense_layer, activation, loss_function, X_batch, y_batch, optimizer, delta):
    with tf.GradientTape() as tape:
        # Forward pass with delta constraint
        dense_layer.forward(X_batch, delta=delta)
        activation.forward(dense_layer.output)
        loss = loss_function.calculate(activation.output, y_batch)

    # Compute gradients
    gradients = tape.gradient(loss, [dense_layer.weights, dense_layer.Gweights, dense_layer.bias])

    # Ensure valid gradients
    if gradients is None or any(g is None for g in gradients):
        return None  # If gradient calculation fails, return None

    # Apply gradients
    optimizer.apply_gradients(zip(gradients, [dense_layer.weights, dense_layer.Gweights, dense_layer.bias]))

    # Compute accuracy
    predictions = tf.cast(activation.output > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predictions, y_batch), dtype=tf.float32))

    return loss, accuracy

In [6]:
def train_custom_nn(X_train, y_train, epochs=5, batch_size=32, learning_rate=0.01, delta=0.5):
    num_samples = X_train.shape[0]

    # Convert input data & labels to TensorFlow tensors
    X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_train_tf = tf.convert_to_tensor(y_train.reshape(-1, 1), dtype=tf.float32)

    # Create TensorFlow dataset for efficient training
    dataset = tf.data.Dataset.from_tensor_slices((X_train_tf, y_train_tf))
    dataset = dataset.shuffle(num_samples).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    # Initialize custom model
    with tf.device(device):
        dense_layer = CustomDenseLayerTF(X_train.shape[1], 1)
        activation = ActivationSigmoidTF()
        loss_function = LossBinaryCrossentropyTF()
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Training loop
    for epoch in range(epochs):
        total_loss = tf.Variable(0.0, dtype=tf.float32)
        total_accuracy = tf.Variable(0.0, dtype=tf.float32)
        num_batches = tf.Variable(0, dtype=tf.int32)

        # Process data in batches using TensorFlow dataset
        for X_batch, y_batch in dataset:
            result = train_step(dense_layer, activation, loss_function, X_batch, y_batch, optimizer, delta)

            if result is None:
                continue  # Skip this batch if train_step() returned None

            loss, accuracy = result
            total_loss.assign_add(loss)
            total_accuracy.assign_add(accuracy)
            num_batches.assign_add(1)

        # Compute average loss and accuracy per epoch
        avg_loss = total_loss / tf.cast(num_batches, tf.float32)
        avg_accuracy = total_accuracy / tf.cast(num_batches, tf.float32)

        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss.numpy():.4f}, Accuracy: {avg_accuracy.numpy():.4f}")

    return dense_layer, activation

In [7]:
# Train the custom neural network with a batch size of 64
dense_layer, activation = train_custom_nn(x_train, y_train_even_odd, epochs=10, batch_size=32, delta=0.2)


2025-02-28 11:44:19.255468: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-02-28 11:44:19.255506: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-28 11:44:19.255514: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-28 11:44:19.255532: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-28 11:44:19.255548: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-02-28 11:44:19.566236: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-02-28 11:44:21.165318: I t

Epoch 1/10 - Loss: 0.3072, Accuracy: 0.8705


2025-02-28 11:44:22.189119: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2/10 - Loss: 0.2736, Accuracy: 0.8919
Epoch 3/10 - Loss: 0.2702, Accuracy: 0.8961


2025-02-28 11:44:24.220999: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 4/10 - Loss: 0.2647, Accuracy: 0.8985
Epoch 5/10 - Loss: 0.2663, Accuracy: 0.8980
Epoch 6/10 - Loss: 0.2691, Accuracy: 0.8979
Epoch 7/10 - Loss: 0.2654, Accuracy: 0.8981


2025-02-28 11:44:28.350312: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 8/10 - Loss: 0.2660, Accuracy: 0.8971
Epoch 9/10 - Loss: 0.2604, Accuracy: 0.9018
Epoch 10/10 - Loss: 0.2657, Accuracy: 0.8961
